### 따로따로 실행해보고 마지막에 하나의 함수로 합치는 식으로 하겠음

In [2]:
import os
import warnings
import sys

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

import mlflow
import mlflow.sklearn
import logging

logging.basicConfig(level = logging.WARN)
logger = logging.getLogger(__name__)


In [3]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

warnings.filterwarnings('ignore')
np.random.seed(40)

In [7]:
csv_url = "http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
try:
    data = pd.read_csv(csv_url, sep = ';')
except Exception as e:
    logger.exception(
    "대충 못 받아왔다는 뜻")

In [8]:
train, test = train_test_split(data)

train_x = train.drop(['quality'], axis = 1)
test_x = test.drop(['quality'], axis = 1)
train_y = train[['quality']]
test_y = test[['quality']]

In [13]:
# 여긴 인풋 받는 공간임
in_alpha = 0.5
in_l1_ratio = 0.5

if float(in_alpha) is None:
    alpha = 0.5
else : 
    alpha = float(in_alpha)
    
if float(in_l1_ratio) is None:
    l1_ratio = 0.5
else:
    l1_ratio = float(in_l1_ratio)

In [14]:
with mlflow.start_run():
    lr = ElasticNet(alpha = alpha, l1_ratio = l1_ratio, random_state = 42)
    lr.fit(train_x, train_y)
    
    predicted_qualities = lr.predict(test_x)
    (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)
    
    # Print out metrics
    print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
    print("  RMSE: %s" % rmse)
    print("  MAE: %s" % mae)
    print("  R2: %s" % r2)

    # mlflow에 파라미터와 수치 값을 기록 - 이 과정이 좀 걸림
    mlflow.log_param("alpha", alpha)
    mlflow.log_param("l1_ratio", l1_ratio)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)
    mlflow.log_metric("mae", mae)

    # mlflow에 모델 기록
    mlflow.sklearn.log_model(lr, "model")

Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.7931640229276851
  MAE: 0.6271946374319586
  R2: 0.10862644997792614


In [16]:
# 위의 함수 내용 통합
def train(in_alpha, in_l1_ratio):
    import os
    import warnings
    import sys

    import pandas as pd
    import numpy as np
    from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
    from sklearn.model_selection import train_test_split
    from sklearn.linear_model import ElasticNet

    import mlflow
    import mlflow.sklearn

    import logging

    logging.basicConfig(level=logging.WARN)
    logger = logging.getLogger(__name__)

    def eval_metrics(actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2

    warnings.filterwarnings("ignore")
    np.random.seed(40)

    # Read the wine-quality csv file from the URL
    csv_url = (
        "http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
    )
    try:
        data = pd.read_csv(csv_url, sep=";")
    except Exception as e:
        logger.exception(
            "Unable to download training & test CSV, check your internet connection. Error: %s", e
        )

    # Split the data into training and test sets. (0.75, 0.25) split.
    train, test = train_test_split(data)

    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]

    # Set default values if no alpha is provided
    if float(in_alpha) is None:
        alpha = 0.5
    else:
        alpha = float(in_alpha)

    # Set default values if no l1_ratio is provided
    if float(in_l1_ratio) is None:
        l1_ratio = 0.5
    else:
        l1_ratio = float(in_l1_ratio)

    # Useful for multiple runs (only doing one run in this sample notebook)
    with mlflow.start_run():
        # Execute ElasticNet
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        # Evaluate Metrics
        predicted_qualities = lr.predict(test_x)
        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        # Print out metrics
        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        # Log parameter, metrics, and model to MLflow
        mlflow.log_param("alpha", alpha)
        mlflow.log_param("l1_ratio", l1_ratio)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)

        mlflow.sklearn.log_model(lr, "model")

train(0.1, 0.1)

Elasticnet model (alpha=0.100000, l1_ratio=0.100000):
  RMSE: 0.7128829045893679
  MAE: 0.5462202174984664
  R2: 0.2799376066653344


### 튜토리얼은 train.py를 따라간다

`train.py`는 다르게 작성해야 하므로(`if __name__ == "__main__":` 구문이 들어가기 때문에) 약간 다르게 작성해야 함

`python train.py`로 실행하면 되는데, `pyhton train.py <alpha> <l1_ratio>`로도 실행할 수 있음.  
- 후자가 가능한 이유는 `train.py` 내부에 `sys.argv[1]`, `sys.argv[2]` 어쩌구가 명시되어 있기 때문으로, `sys.argv[0]`은 해당 파일의 이름 이고 1번째 argument부터는 위처럼 함수 내부에 넣어줄 수 있음(물론 그 인자를 처리하는 내용이 들어가야겠죠?)

In [17]:
!python train.py

Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.7931640229276851
  MAE: 0.6271946374319586
  R2: 0.10862644997792614


In [18]:
!python train.py 0.2 0.2

Elasticnet model (alpha=0.200000, l1_ratio=0.200000):
  RMSE: 0.7336400911821402
  MAE: 0.5643841279275428
  R2: 0.23739466063584158


## 모델 비교하기

In [19]:
!mlflow ui # http://localhost:5000 에서 볼 수 있다

^C


- 마우스로 정렬을 제어할 수 있고, 검색어에 `metrics.rmse < 0.8` 같은 조건을 넣어서 검색할 수도 있다.
- 더 정교한 조작을 위해, 이 모델들의 결과 자체를 csv 파일로 다운받을 수도 있다.

## Conda 환경에 훈련 코드 패키징하기

훈련 코드를 패키지해서 다른 사람이나 환경에서 쉽게 재사용할 수 있게 할 수 있다.  
`MLflow Projects`를 이용해 의존성과 엔트리 포인트를 지정할 수 있다.
- 예제의 MLProject 파일은 이렇다
- `MLproject` (확장자명이 따로 없음)
```yaml
name: tutorial

python_env: python_env.yaml

entry_points:
  main:
    parameters:
      alpha: {type: float, default: 0.5}
      l1_ratio: {type: float, default: 0.1}
    command: "python train.py {alpha} {l1_ratio}"
```

- `entry_points` : 프로젝트 내에서 실행할 수 있는 명령과 매개변수에 관한 정보로, 대부분의 프로젝트에서 `entry_points`는 1개 이상 포함되어 있다.
    - 프로젝트의 `.py` 또는 `.sh` 파일을 엔트리 포인트로 호출할 수 있다.
    - `MLProject` 파일에 진입점을 나열한다면 데이터 유형, 기본값을 포함한 매개변수(`parameter`)를 지정할 수도 있다.
    
- `environment` : `entry_points`를 실행하는 데 필요한 소프트웨어 환경으로, 모든 라이브러리 종속성이 포함된다.
     - `Conda` 환경, `venv` 환경, `Docker` 컨테이너 등이 사용 가능함
 
 - `$ mlflow run`이나 `mlflow.projects.run()`을 사용한 깃 URL이나 로컬 디렉토리에서 어떤 프로젝트든 사용 가능하다.
     - 일반적으로 Git 프로젝트에 대해 새로운 임시 작업 디렉토리를 사용하기 때문에 모든 파일 인자는 **절대 경로**로 전달되어야 한다.

예제에서는 `conda.yaml`와 `python_env.yaml`을 지정했는데, 각각의 내용은
- `conda.yaml`
```yaml
name: tutorial
channels:
  - conda-forge
dependencies:
  - python=3.8
  - pip
  - pip:
      - scikit-learn==0.23.2
      - mlflow>=1.0
      - pandas
```
- `python_env.yaml`
```yaml
python: "3.8"
build_dependencies:
  - pip
dependencies:
  - scikit-learn==0.23.2
  - mlflow>=1.0
  - pandas
```

- 실제로는 MLproject에서 python_env만 지정했기 때문에 conda는 없어도 될 것 같음
- 중요한 건 `yaml`을 어떻게 작성하느냐일 것 같음 : 전부 자동으로 작성하면 이 파일이 너무 복잡해짐

- 이렇게 명시해둔 프로젝트를 실행해본다
- 튜토리얼에서 `mlflow run sklearn.elasticnet_wine -P alpha=0.42`라고 되어있는 걸 봐선 경로 하나를 올라가서 실행시켜봐야겠음

In [26]:
# 이 명령어는 깃에 접속하는 명령어 같단 말임?
# !mlflow run sklearn_elasticnet_wine -P alpha=0.42 # 얘는 접근이 안된다며 실행 못함

# `MLproject` 파일이 root에 있다면 깃허브에서 직접 가져와서 실행할 수 있다고 함
!mlflow run https://github.com/mlflow/mlflow-example.git -P alpha=5.0

Traceback (most recent call last):
  File "C:\Users\dowra\anaconda3\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\dowra\anaconda3\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\dowra\anaconda3\Scripts\mlflow.exe\__main__.py", line 7, in <module>
  File "C:\Users\dowra\anaconda3\lib\site-packages\click\core.py", line 1128, in __call__
    return self.main(*args, **kwargs)
  File "C:\Users\dowra\anaconda3\lib\site-packages\click\core.py", line 1053, in main
    rv = self.invoke(ctx)
  File "C:\Users\dowra\anaconda3\lib\site-packages\click\core.py", line 1659, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "C:\Users\dowra\anaconda3\lib\site-packages\click\core.py", line 1395, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "C:\Users\dowra\anaconda3\lib\site-packages\click\core.py", line 754, in invoke
    return __callback(*args, **kwargs)


## pip requirements 명시하기 : `pip_requirements`, `extra_pip_requirements`
- 이거 실행은 못함! 어떻게 돌아가는지 파악하는 용도로 볼 것

In [29]:
import tempfile

import sklearn
from sklearn.datasets import load_iris
import xgboost as xgb
import mlflow
from mlflow.tracking import MlflowClient
from mlflow.artifacts import download_artifacts


def read_lines(path):
    with open(path) as f:
        return f.read().splitlines()


def get_pip_requirements(run_id, artifact_path, return_constraints=False):
    client = MlflowClient()
    req_path = download_artifacts(run_id=run_id, artifact_path=f"{artifact_path}/requirements.txt")
    reqs = read_lines(req_path)

    if return_constraints:
        con_path = download_artifacts(
            run_id=run_id, artifact_path=f"{artifact_path}/constraints.txt"
        )
        cons = read_lines(con_path)
        return set(reqs), set(cons)

    return set(reqs)


def main():
    iris = load_iris()
    dtrain = xgb.DMatrix(iris.data, iris.target)
    model = xgb.train({}, dtrain)

    xgb_req = f"xgboost=={xgb.__version__}"
    sklearn_req = f"scikit-learn=={sklearn.__version__}"

    with mlflow.start_run() as run:
        run_id = run.info.run_id

        # Default (both `pip_requirements` and `extra_pip_requirements` are unspecified)
        artifact_path = "default"
        mlflow.xgboost.log_model(model, artifact_path)
        pip_reqs = get_pip_requirements(run_id, artifact_path)
        assert pip_reqs.issuperset(["mlflow", xgb_req]), pip_reqs

        # Overwrite the default set of pip requirements using `pip_requirements`
        artifact_path = "pip_requirements"
        mlflow.xgboost.log_model(model, artifact_path, pip_requirements=[sklearn_req])
        pip_reqs = get_pip_requirements(run_id, artifact_path)
        assert pip_reqs == {"mlflow", sklearn_req}, pip_reqs

        # Add extra pip requirements on top of the default set of pip requirements
        # using `extra_pip_requirements`
        artifact_path = "extra_pip_requirements"
        mlflow.xgboost.log_model(model, artifact_path, extra_pip_requirements=[sklearn_req])
        pip_reqs = get_pip_requirements(run_id, artifact_path)
        assert pip_reqs.issuperset(["mlflow", xgb_req, sklearn_req]), pip_reqs

        # Specify pip requirements using a requirements file
        with tempfile.NamedTemporaryFile("w", suffix=".requirements.txt") as f:
            f.write(sklearn_req)
            f.flush()

            # Path to a pip requirements file
            artifact_path = "requirements_file_path"
            mlflow.xgboost.log_model(model, artifact_path, pip_requirements=f.name)
            pip_reqs = get_pip_requirements(run_id, artifact_path)
            assert pip_reqs == {"mlflow", sklearn_req}, pip_reqs

            # List of pip requirement strings
            artifact_path = "requirements_file_list"
            mlflow.xgboost.log_model(
                model, artifact_path, pip_requirements=[xgb_req, f"-r {f.name}"]
            )
            pip_reqs = get_pip_requirements(run_id, artifact_path)
            assert pip_reqs == {"mlflow", xgb_req, sklearn_req}, pip_reqs

        # Using a constraints file
        with tempfile.NamedTemporaryFile("w", suffix=".constraints.txt") as f:
            f.write(sklearn_req)
            f.flush()

            artifact_path = "constraints_file"
            mlflow.xgboost.log_model(
                model, artifact_path, pip_requirements=[xgb_req, f"-c {f.name}"]
            )
            pip_reqs, pip_cons = get_pip_requirements(
                run_id, artifact_path, return_constraints=True
            )
            assert pip_reqs == {"mlflow", xgb_req, "-c constraints.txt"}, pip_reqs
            assert pip_cons == {sklearn_req}, pip_cons


if __name__ == "__main__":
    main()

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\dowra\\AppData\\Local\\Temp\\tmp8xh4ac1t.requirements.txt'

# 모델 배포하기
- MLflow Model은 Batch / Streaming 추론 모두를 지원하기 위해 모델을 패키징하는 표준 형식이다

In [32]:
# 이 예제에서 모델은 run 내의 artifact로 저장된다.
mlflow.sklearn.log_model(lr, 'model') 

In [33]:
!mlflow ui

^C


- 실행시킨 모델을 클릭하고 들어가서 쭉 내려서 `Artifacts` 란을 보자
- `mlflow.sklearn.log_model`에 의해 생성된 파일들

    1. `MLmodel`의 경로를 보면 대충 아래처럼 지정되어 있다.
    `file:///C:/Users/dowra/1.MyWorkspace/MLFlow/examples/sklearn_elasticnet_wine/mlruns/0/11e99a0472a34e0b804a8f8a8af07131/artifacts/model/MLmodel`
        - 이 파일은 로, `MLflow`가 모델을 어떻게 불러왔는지를 설명해준다.

    2. `model.pkl` : 사용자가 학습한 선형 회귀 모델의 직렬화된 버전


#### 모델을 MLflow 서버와 함께 사용해 예측을 지원하는 로컬 REST 서버를 배포하기
- 참고 ) 모델 생성에 사용된 파이썬 버전은 mlflow 모델 실행 버전과 동일해야 한다. 아니라면 오류가 발생할 수 있음.

In [34]:
!mlflow models serve -m C:/Users/dowra/1.MyWorkspace/MLFlow/examples/sklearn_elasticnet_wine/mlruns/0/11e99a0472a34e0b804a8f8a8af07131/artifacts/model -p 1234

2022/12/14 16:34:55 INFO mlflow.models.flavor_backend_registry: Selected backend for flavor 'python_function'
Traceback (most recent call last):
  File "C:\Users\dowra\anaconda3\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\dowra\anaconda3\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\dowra\anaconda3\Scripts\mlflow.exe\__main__.py", line 7, in <module>
  File "C:\Users\dowra\anaconda3\lib\site-packages\click\core.py", line 1128, in __call__
    return self.main(*args, **kwargs)
  File "C:\Users\dowra\anaconda3\lib\site-packages\click\core.py", line 1053, in main
    rv = self.invoke(ctx)
  File "C:\Users\dowra\anaconda3\lib\site-packages\click\core.py", line 1659, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "C:\Users\dowra\anaconda3\lib\site-packages\click\core.py", line 1659, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File 